# Instalação

In [ ]:
import tensorflow as tf
import pkg_resources

required_version = '2.12.0'

# Verificar se o TensorFlow já está instalado e obter a versão
try:
	version = pkg_resources.get_distribution("tensorflow").version
except pkg_resources.DistributionNotFound:
	version = None

# Verificar a versão e instalar se necessário
if version is None or version != required_version:
	print(f"Instalando TensorFlow {required_version}...")
	!pip install tensorflow=={required_version}
else:
	print(f"Versão do TensorFlow {version} já está instalada.")


In [ ]:
import requests
import pandas as pd
import numpy as np


# Dados Treino Herácles (entrada 1) 

In [ ]:
diaInicio = '20210301'
diaFinal = '20220831'
idGrupo = '12'
idRegra = '125'
idTipo = '0'
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreinoEntradaSaida?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(url)


In [ ]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)
df.head()


In [ ]:
dfTratadoX = df['campoX'].str.split(',').apply(lambda x: [float(val) for val in x])
Entrada1 = np.stack(dfTratadoX.values)
Entrada1.shape


In [ ]:
Y = np.stack(df['campoY'].values)
Saida1 = Y.reshape(-1,1).astype(int)
Saida1.shape


In [ ]:
# Verificar se há valores não numéricos
has_non_numeric = np.isnan(Entrada1).any() or np.isinf(Entrada1).any()
if has_non_numeric:
    print("Entrada1 array contém valores não numéricos.")
else:
    print("Entrada1 contém apenas valores numéricos.")


In [ ]:
# Verificar se há valores não numéricos
has_non_numeric = np.issubdtype(Saida1.dtype, np.number) is False
if has_non_numeric:
    print("Saida1 contém valores não numéricos.")
else:
    print("Saida1 contém apenas valores numéricos.")


# Minha Rede

## Estrutra

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units=64, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True, input_shape=(300,1)),
    tf.keras.layers.LSTM(units=32, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True),
    tf.keras.layers.LSTM(units=16, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform()),
    tf.keras.layers.Dense(units=1, activation='sigmoid', kernel_initializer=tf.keras.initializers.HeUniform())
])
model.summary()


## Compilacao

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy',  metrics=['accuracy'])


## Fit - Treino

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)

X2 = X.T
X2 = X2[np.newaxis,...]
history = model.fit(x=X2.T,y=Y,epochs=80,verbose=2,callbacks=[callback])


## Salva Rede

In [ ]:
path='pesos/treino_rede743.h5'
model.save(path)


# Enviando Rede

In [ ]:
!pip install paramiko


In [ ]:
import os
import paramiko


In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("191.252.219.136", username="root", password="Digicom2004$")
sftp = ssh.open_sftp()
sftp.put("pesos/treino_rede743.h5", "/root/executa-rede/redes/treino_rede743.h5")
sftp.close()
ssh.close()


# Registra Treinamento

In [ ]:
#import requests
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/executouTreinamento'
myobj = {'id': 743}
x = requests.post(url, json = myobj)
print(x.text)
